# Importing all libraries

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import sklearn
import nltk
import pickle as pkl

In [2]:
# making dataframes from datasets
movie=pd.read_csv("tmdb_5000_movies.csv")
credit=pd.read_csv("tmdb_5000_credits.csv")

# Checking the data


In [3]:
movies=movie.merge(credit,on="title")

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

# Column selection 

In [5]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [6]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
movies.drop(columns=['budget',"homepage","original_language","original_title","popularity","production_companies","production_countries",'release_date','revenue','runtime',
       'spoken_languages', 'status', 'tagline','vote_count', 'movie_id'],inplace=True)

In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        4809 non-null   object 
 1   id            4809 non-null   int64  
 2   keywords      4809 non-null   object 
 3   overview      4806 non-null   object 
 4   title         4809 non-null   object 
 5   vote_average  4809 non-null   float64
 6   cast          4809 non-null   object 
 7   crew          4809 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 300.7+ KB


# Preprocessing

In [9]:
movies.isnull().sum()  # check null rows

genres          0
id              0
keywords        0
overview        3
title           0
vote_average    0
cast            0
crew            0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
movies.duplicated().sum() #check duplicated rows

0

In [12]:
movies=movies[["id","title","genres","keywords","overview","cast","crew","vote_average"]] #rearranging the columns for better inntuitive dataset

#### 1. Checking the genre column

In [13]:
movies.genres[0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
# make a function to get the genres in list format
def get_genres(obj):
    genre_list=[]
    for i in ast.literal_eval(obj):  #literal eval converts string to list by raising an exception when datatype isnt valid
        genre=i["name"]
        genre_list.append(genre)
    return genre_list

In [15]:
# check if get_genres is working properly
get_genres(movies.genres[0])

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [16]:
movies.genres=movies.genres.apply(get_genres)

#### 2. Checking the keyword column

In [17]:
movies.keywords[0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [18]:
movies.keywords=movies.keywords.apply(get_genres)

#### 3. Checking cast and crew column

In [19]:
ast.literal_eval(movies.cast[0])[0]

{'cast_id': 242,
 'character': 'Jake Sully',
 'credit_id': '5602a8a7c3a3685532001c9a',
 'gender': 2,
 'id': 65731,
 'name': 'Sam Worthington',
 'order': 0}

In [20]:
ast.literal_eval(movies.cast[0])[0].keys()

dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order'])

In [21]:
# make a function to get the actor names in list format
def get_names(obj):
    names_list=[]
    j=0
    for i in ast.literal_eval(obj):  #literal eval converts string to list
        names=i["name"]
        if j!=3:
            names_list.append(names)
            j+=1
        else:
            break        
    return names_list

In [22]:
# check if get_names is working properly
get_names(movies.cast[0])

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [23]:
movies.cast=movies.cast.apply(get_names)

In [24]:
ast.literal_eval(movies.crew[0])[0]

{'credit_id': '52fe48009251416c750aca23',
 'department': 'Editing',
 'gender': 0,
 'id': 1721,
 'job': 'Editor',
 'name': 'Stephen E. Rivkin'}

In [25]:
ast.literal_eval(movies.crew[0])[0].keys()

dict_keys(['credit_id', 'department', 'gender', 'id', 'job', 'name'])

In [26]:
# make a function to get the director names in list format
def get_director(obj):
    names_list=[]
    for i in ast.literal_eval(obj):  #literal eval converts string to list
        names=i["name"]
        if i["job"]=="Director":
            names_list.append(names)
            break
        else:
            continue        
    return names_list

In [27]:
movies['director']=''
movies.director=movies.crew.apply(get_director)

#### 4. Checking the overview column

In [28]:
movies.overview=movies.overview.apply(lambda x:x.split())

#### 5. Creating finalized dataset

In [29]:
def remove_spaces(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [30]:
movies.director

0           [James Cameron]
1          [Gore Verbinski]
2              [Sam Mendes]
3       [Christopher Nolan]
4          [Andrew Stanton]
               ...         
4804     [Robert Rodriguez]
4805         [Edward Burns]
4806          [Scott Smith]
4807          [Daniel Hsia]
4808     [Brian Herzlinger]
Name: director, Length: 4806, dtype: object

In [31]:
# Removing the spacces so that director names and actor names are treated as a single entity (eg: Leo and dicaprio aren't treated as separate words, rather Leodicaprio is treated as a single word)
movies['cast'] = movies['cast'].apply(remove_spaces)
movies['director'] = movies['director'].apply(remove_spaces)
movies['genres'] = movies['genres'].apply(remove_spaces)
movies['keywords'] = movies['keywords'].apply(remove_spaces)

In [32]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['director']

In [33]:
movies.drop(columns=['genres', 'keywords', 'overview', 'cast', 'crew','director'], inplace=True)

In [34]:
movies.columns

Index(['id', 'title', 'vote_average', 'tags'], dtype='object')

In [35]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x)).apply(lambda x: x.lower())  # Convert list to string and all letters to lowercase

In [36]:
movies.tags[0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [37]:
movies.head()

,id,title,vote_average,tags
0,19995,Avatar,7.2,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,6.9,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,6.3,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,7.6,following the death of district attorney harve...
4,49529,John Carter,6.1,"john carter is a war-weary, former military ca..."


# Making the model

## Stemming

In [39]:
#many words are repeated like {action,actions},{actor,actors}, thus stemming is needed
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [53]:
def stem_words(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    str=" ".join(l)
    return str

In [54]:
movies.tags=movies.tags.apply(stem_words)

## Vectorization

In [55]:
# Vectorisation of words by converting into bag of words and choosing 5000 most common
from sklearn.feature_extraction.text import CountVectorizer

In [80]:
 cv=CountVectorizer(max_features=2000,stop_words="english")  # hyperparameter max_features to be tweaked as per performance

In [81]:
cv.fit_transform(movies.tags).toarray().shape

(4806, 2000)

In [82]:
cv.get_feature_names_out()

array(['000', '10', '11', ..., 'youth', 'zombi', 'zoo'], dtype=object)

In [83]:
vector_movies=cv.fit_transform(movies.tags).toarray()

## Similarity

In [84]:
# Calculate similarity between each vector
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
similar=cosine_similarity(vector_movies)

In [86]:
similar.shape

(4806, 4806)

## Recommender

In [87]:
def recommend_mov(movie):
    movie_index=movies[movies.title==movie].index[0]
    distance=similar[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        movie_names=movies.iloc[i[0]].title
        print(movie_names)
    
    return 

In [88]:
recommend_mov("Batman Begins")

The Dark Knight
Batman
Batman
10th & Wolf
Superman


# Exporting file to pycharm

In [96]:
pkl.dump(movies,open("movies_list.pkl","wb"))

In [95]:
pickle.dump(similar,open('similarity.pkl','wb'))